# Wiring inputs and outputs with TableReporter

- This example shows how to wire inputs and outputs by reporting the position of the system's center of mass.
- We also illustrate that input-output connections are stored in model (.osim) files.
- The model contains just one body, a free joint, and the table reporter.

In [1]:
import opensim as osim

In [2]:
model_filename = 'wiring_inputs_and_outputs_with_TableReporter.osim'

This function create and prints the model to a `.osim` file

In [3]:
def print_model():
    model = osim.Model()
    model.setName('model')
    
    # Create a body with name 'body', mass of 1 kg, center of mass at the
    # origin of the body, and unit inertia (Ixx = Iyy = Izz = 1 kg-m^2).
    body = osim.Body('body', 1.0, osim.Vec3(0), osim.Inertia(1))
    
    # Create a free joint (all 6 degrees of freedom) with Ground as the parent
    # body and 'body' as the child body.
    joint = osim.FreeJoint('joint', model.getGround(), body)
    
    # Add the body and joint to the model.
    model.addComponent(body)
    model.addComponent(joint)
    
    # Create a TableReporter to save quantities to a file after simulating.
    reporter = osim.TableReporterVec3()
    reporter.setName('reporter')
    reporter.set_report_time_interval(0.1)
    # Report the position of the origin of the body.
    reporter.addToReport(body.getOutput('position'))
    # For comparison, we will also get the center of mass position from the
    # Model, and we can check that the two outputs are the same for our
    # one-body system. The (optional) second argument is an alias for the name
    # of the output; it is used as the column label in the table.
    reporter.addToReport(model.getOutput('com_position'), 'com_pos')
    # Display what input-output connections look like in XML (in .osim files).
    print("Reporter input-output connections in XML:\n" + reporter.dump())
    
    model.addComponent(reporter)

    model.printToXML(model_filename)

Create and print the model file

In [4]:
print_model()

Reporter input-output connections in XML:
<TableReporter__Vec3_>
    <!--List of components that this component owns and serializes.-->
    <components />
    <!--The recording time interval (s). If interval == 0, defaults toevery valid integration time step.-->
    <report_time_interval>0.10000000000000001</report_time_interval>
    <!--Paths to outputs (channels) to satisfy the list Input 'inputs' of type InputT (description: Variable list of quantities to be reported.). To specify multiple paths, put spaces between them.-->
    <input_inputs_connectee_names>../model/body|position ../model|com_position(com_pos)</input_inputs_connectee_names>
</TableReporter__Vec3_>



Load the model file

In [7]:
deserialized_model = osim.Model(model_filename)
state = deserialized_model.initSystem()

We can fetch the TableReporter from within the deserialized model

In [8]:
reporter = osim.TableReporterVec3.safeDownCast(
        deserialized_model.getComponent('reporter'))

We can access the names of the outputs that the reporter is connected to

In [9]:
print('Outputs connected to the reporter:')
for i in range(reporter.getInput('inputs').getNumConnectees()):
    print(reporter.getInput('inputs').getConnecteeName(i))

Outputs connected to the reporter:
../body|position
..|com_position(com_pos)


Simulate the model

In [10]:
manager = osim.Manager(deserialized_model)
manager.setInitialTime(0)
manager.setFinalTime(1.0)
manager.integrate(state)

True

Now that the simulation is done, get the table from the TableReporter and write it to a file
- This returns the TimeSeriesTableVec3 that holds the history of positions.

In [11]:
table = reporter.getTable()

Create a FileAdapter, which handles writing to (and reading from) .sto files

In [12]:
sto = osim.STOFileAdapterVec3()
sto.write(table, 'wiring_inputs_and_outputs_with_TableReporter.sto')